In [113]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [114]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [115]:
include("global_adder_passthrough.jl")

filter_highest_tag_number (generic function with 1 method)

In [116]:
# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 12
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = productMPS(qubit_site, [1,2,1,2,2,1,2,1,2,1,2,1,1,1,2,2])
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# initialize tao adder mpo
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
initial_state_1 = copy(initial_state)
initial_state_2 = copy(initial_state)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 12, 2, 11, 3, 10, 4, 9, 5, 8, 6, 7]
[1, 3, 5, 7, 9, 11, 12, 10, 8, 6, 4, 2]


MPS
[1] ((dim=2|id=955|"Qubit,Site,n=1"), (dim=50|id=296|"Link,l=1"))
[2] ((dim=50|id=296|"Link,l=1"), (dim=2|id=413|"Qubit,Site,n=2"), (dim=50|id=624|"Link,l=2"))
[3] ((dim=50|id=624|"Link,l=2"), (dim=2|id=541|"Qubit,Site,n=3"), (dim=50|id=581|"Link,l=3"))
[4] ((dim=50|id=581|"Link,l=3"), (dim=2|id=503|"Qubit,Site,n=4"), (dim=50|id=593|"Link,l=4"))
[5] ((dim=50|id=593|"Link,l=4"), (dim=2|id=178|"Qubit,Site,n=5"), (dim=50|id=438|"Link,l=5"))
[6] ((dim=50|id=438|"Link,l=5"), (dim=2|id=699|"Qubit,Site,n=6"), (dim=50|id=677|"Link,l=6"))
[7] ((dim=50|id=677|"Link,l=6"), (dim=2|id=379|"Qubit,Site,n=7"), (dim=32|id=548|"Link,l=7"))
[8] ((dim=32|id=548|"Link,l=7"), (dim=2|id=398|"Qubit,Site,n=8"), (dim=16|id=567|"Link,l=8"))
[9] ((dim=16|id=567|"Link,l=8"), (dim=2|id=565|"Qubit,Site,n=9"), (dim=8|id=164|"Link,l=9"))
[10] ((dim=8|id=164|"Link,l=9"), (dim=2|id=797|"Qubit,Site,n=10"), (dim=4|id=855|"Link,l=10"))
[11] ((dim=4|id=855|"Link,l=10"), (dim=2|id=357|"Qubit,Site,n=11"), (dim=2|id=570|"L

In [117]:
println(contract(initial_state_1).tensor)
println(contract(initial_state_2).tensor)

[-0.015519249533979337 -0.014587932351547242; -0.004825796122411103 0.01027119938690027;;; 0.015872332507184488 -0.00799969527012161; 0.014663145118745405 0.003220188341815563;;;; -0.0017290626842539592 0.01833418554225222; 0.011849154841695714 0.009234584641645404;;; -0.006423832087079831 0.015354214567973869; -0.022674109397385643 -0.009972056022754281;;;;; 0.015266022445451053 0.01188571549124635; 0.0011040648272995174 0.008328386531695011;;; 0.0037874423293522046 0.002725849565639738; -0.002787864781482628 0.006121703146995543;;;; 0.0067016442686932045 0.00908205017961121; -0.005047332044324004 0.013431530924810958;;; 0.027989061315236558 -0.0027753626867406543; 0.010098776875288576 -0.009084331239766775;;;;;; 0.01923628013353785 0.015052512279817054; 0.049250408076023836 0.022991755890007876;;; -0.0011951241655157837 -0.0006441127862580425; -0.00939387586786813 0.01402463888387766;;;; -0.0010987119394312169 -0.008670067289023529; 0.010927504572587888 0.003449069000907607;;; 0.0079

In [118]:
initial_state_1 = apply(add1_3,initial_state_1)
initial_state_2 = global_adder(initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)
println(contract(initial_state_1).tensor)
println(contract(initial_state_2).tensor)

[0.02546535929064335 -0.012446612961769354; 0.0002940212944938753 0.0035279949240717137;;; -0.013983166152862473 0.019211590417600535; -0.005289944740904338 0.000438702588836965;;;; -0.02434193688892383 -0.023478291061475233; -0.0025728563455852007 -0.006547773197984992;;; -0.0014575688160536069 0.012212542889964855; 0.01562058941124708 -0.0015356314833119711;;;;; -0.002079174739115287 -0.0020184722062577868; 0.02824938179195339 -0.020161503880623873;;; -0.0014501018325697862 0.013121022070970476; 0.0029036574062324175 -0.004593474741967446;;;; 0.007202711735933815 0.006888599299011965; -0.007801450560088664 -0.007230819679684374;;; 0.0050539429947188 -0.013448804249158095; -0.017390971251014554 0.00027675485463587296;;;;;; -0.025877746002291917 -0.007939806268665401; 0.014608454545230475 -0.0026734807465995363;;; -0.00452445298786189 -0.0090247326140296; -0.008509302129478955 0.0053626322524799545;;;; 0.01691555277328093 0.0025651275486688837; -0.007155249488338859 0.00475005658286459

In [119]:
using BenchmarkTools
BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
BenchmarkTools.DEFAULT_PARAMETERS.evals = 1
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 300


300

In [120]:
@benchmark initial_state_1 = apply(add1_3,$initial_state_1)


BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  10.754 ms … 31.638 ms  ┊ GC (min … max): 0.00% … 9.39%
 Time  (median):     14.927 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.746 ms ±  4.741 ms  ┊ GC (mean ± σ):  4.31% ± 7.06%

    ▃   █▃▃▁ ▄▁          ▁                                     
  ▄▆█▆▆▆████▄██▄▇▁▇▄▄▆▆▁▇█▆▆▄▁▄▄▁▆▄▄▆▁▆▄▄▁▄▁▆▄▄▆▁▄▁▄▄▆▁▁▁▁▄▁▆ ▄
  10.8 ms         Histogram: frequency by time        27.9 ms <

 Memory estimate: 18.15 MiB, allocs estimate: 12583.

In [121]:
@benchmark initial_state_2 = global_adder($initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  1.695 s …    7.848 s  ┊ GC (min … max): 0.48% … 0.88%
 Time  (median):     2.215 s               ┊ GC (median):    0.76%
 Time  (mean ± σ):   2.555 s ± 910.533 ms  ┊ GC (mean ± σ):  3.30% ± 4.27%

   ▂ █▄▂                                                      
  ▇█▇███▁▇▆▅▇▃▁▃▃▅▃▇▁▁▆▇▆▁▁▆▁▃▁▅▅▃▆▃▁▃▁▁▅▇▁▃▁▁▁▁▁▃▃▃▁▁▁▃▁▁▁▅ ▃
  1.69 s         Histogram: frequency by time         4.55 s <

 Memory estimate: 542.40 MiB, allocs estimate: 13835.

In [ ]:
using ProfileView

In [122]:
BenchmarkTools.DEFAULT_PARAMETERS.seconds

300.0

In [123]:
println(contract(initial_state_2).tensor)

[-0.00110480691571687 0.0155418649932069; 0.00571791851079534 0.004980104418543361;;; 0.009144860279238302 -0.0072022567415999505; 0.011931385027502546 -0.026980481480495103;;;; 0.008050693413112257 -0.0248028333377495; -0.01967979967686029 0.0003541477742075605;;; -0.008262590691811138 -0.007797143293526244; 0.005162000302523061 -0.007797744423932455;;;;; -0.028435457601430864 0.009209724800593646; 0.009407362957522309 -0.002047164007024342;;; 0.013482887076904471 -0.001808681089785708; 0.01925986886540166 0.0032536622509154066;;;; -0.0043095948160132 -0.00973277629637734; -0.0076673813522451836 -0.025548716642632772;;; -0.007264793282231336 0.004514201723944771; -0.0014068337368698784 0.012276241718521546;;;;;; -0.025270664333178077 -0.01829109934779068; 0.02498603279500988 -0.015327247325534735;;; -0.009197450648918004 -0.016789221498001074; -0.02525007599773812 -0.027638463063837847;;;; -0.0026322380400045325 0.013501948591680453; 0.0030787127884063696 0.005984563685458448;;; -0.00

;;; -0.017319708045104017 -0.03020510132939614; 0.00015939563020948223 -0.018544344011409693;;;;; -0.005316214883181333 0.014350807666878103; 0.015882154751411297 0.005595185857642938;;; 0.0010943808898765293 -0.0034449704738489214; 0.013525399199123635 -0.009066560232347434;;;; 0.011354737599872058 0.01935537518970359; 0.014978233902052426 0.00823641896844307;;; 0.012668984291710473 0.016718398506590834; 0.014715650078593856 0.010076881452014777;;;;;;;; -0.00923265014527105 -0.012444812957232973; -0.010698701800947962 -0.016666565751693438;;; -0.025526850546621134 0.01008960346139111; -0.005334003570218601 0.012875843596256485;;;; -0.016593942009680576 0.019337869365552585; 0.01758367483717841 0.004171404139124777;;; 0.003241607618798568 -0.009596766681150882; -0.0014014312940164636 -0.013905850126157466;;;;; 0.008742595754428087 -0.011103171355856393; 0.01008191157326324 0.012062020857843086;;; -0.0008199519402866138 0.004198110153417085; 0.000194710861697635 -0.00513586989291643;;;;

In [124]:
# The `seconds` expression helps set a rough time budget, see Manual for more explanation
@btime sin(x) setup=(x=rand()) seconds=3

  8.600 ns (0 allocations: 0 bytes)


0.7293060492501259

In [125]:
# psi = contract(initial_state)
# println(inds(psi))
# print_nonzero_coordinates(psi)

In [126]:
# function perm(bit_string::String, pattern::Vector{Int})
#     permuted = ['0' for _ in 1:length(bit_string)]
#     for (i, pos) in enumerate(pattern)
#         permuted[pos] = bit_string[i]
#     end
#     return join(permuted)
# end